In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from scripts.pipeline import Pipeline

In [2]:
# Load the data
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

# split train data into predictors and response
X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice'] 

#combine the data for preprocessing
data = pd.concat([X, test_data], axis=0)
# Preprocess the data
data = Pipeline(data)

# Split the data back into train and test, processed
X = data[:len(train_data)]
test = data[len(train_data):]
#increase test index by 1
test.index = test.index + 1

# Split the data into features and response
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# try a few models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

models = [
    LinearRegression(),
    RandomForestRegressor(),
    SVR(),
    GradientBoostingRegressor(),
    AdaBoostRegressor(),
    BaggingRegressor(),
    ExtraTreesRegressor(),
    HistGradientBoostingRegressor(),
]

for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'{model.__class__.__name__} MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'{model.__class__.__name__} R^2: {model.score(X_test, y_test)}')


LinearRegression MSE: 5.401053165269924e+34
LinearRegression R^2: -7.041490830219341e+24
RandomForestRegressor MSE: 882569160.7087352
RandomForestRegressor R^2: 0.8849372064669473
SVR MSE: 7858659144.939162
SVR R^2: -0.024553445663832107
GradientBoostingRegressor MSE: 684781572.7687094
GradientBoostingRegressor R^2: 0.9107232790012155
AdaBoostRegressor MSE: 1259779093.361237
AdaBoostRegressor R^2: 0.835759385020572
BaggingRegressor MSE: 810407467.7019179
BaggingRegressor R^2: 0.8943451105192158
ExtraTreesRegressor MSE: 812837181.5691041
ExtraTreesRegressor R^2: 0.8940283425224507
HistGradientBoostingRegressor MSE: 857279866.6666343
HistGradientBoostingRegressor R^2: 0.8882342362618998
